# Spark

---

Spark le permite distribuir datos y cálculos en clústeres con múltiples nodos (piense en cada nodo como una computadora separada). La división de sus datos facilita el trabajo con conjuntos de datos muy grandes porque cada nodo solo funciona con una pequeña cantidad de datos.

Como cada nodo trabaja en su propio subconjunto de los datos totales, también realiza una parte de los cálculos totales necesarios, de modo que tanto el procesamiento de datos como el cálculo se realizan en paralelo sobre los nodos del clúster. Es un hecho que la computación en paralelo puede hacer que ciertos tipos de tareas de programación sean mucho más rápidas.

El primer paso para usar Spark es conectarse a un clúster. Habrá una computadora, llamada _master_, que administrará la división de los datos y los cálculos. El _master_ está conectado al resto de las computadoras del clúster, que se denominan _worker_. El _master_ envía a los _workers_ datos y cálculos para que se ejecuten, y ellos envían sus resultados al _master_.

## Spark DataFrames

La estructura de datos central de Spark es el Resilient Distributed Dataset (RDD). Este es un objeto de bajo nivel que permite que Spark haga su magia al dividir los datos en varios nodos del clúster. Sin embargo, es difícil trabajar directamente con los RDD, por lo que usaremos Spark DataFrame construido sobre los RDD.

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

### Spark Schema

Si conoce el tipo de dato es bueno definir el schema en Spark, para tener un mejor rendimiento. El schema se puede definir usando la clase `StructType()` que recibe una lista de los diferentes tipos de datos. Para cada variable(o columna) puedo definir el tipo de datos usando la clase `StructField("name", dataType, bool)` que recibe el nombre del campo, el tipo de dato y si los datos pueden ser nulos.

### Data

* mon: mes (entero entre 1 y 12).
* dom: dia del mes (entero entre 1 y 31).
* dow: dia de la semana (1: lunes, ..., 7: domingo).
* org: origen del aeropuerto.
* mile: distancia en millas.
* carrier: operador.
* depart: hora de salida (hora decimal).
* duration: duracion esperada (minutos).
* delay: retraso (minutos).

In [0]:
path_file = "/FileStore/tables/flights.csv"

schema = StructType([
  StructField("mon", IntegerType(), True),
  StructField("dom", IntegerType(), True),
  StructField("dow", IntegerType(), True),
  StructField("carrier", StringType(), True),
  StructField("flight", IntegerType(), True),
  StructField("org", StringType(), True),
  StructField("mile", IntegerType(), True),
  StructField("depart", DoubleType(), True),
  StructField("duration", IntegerType(), True),
  StructField("delay", IntegerType(), True)
])

flights = spark.read.csv(path_file, header=True, schema=schema)
flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5|
 4| 2| 5| AA| 325|ORD| 258| 8.92| 65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

### Lazy evaluation

Spark no realiza ninguna transformación hasta que se solicita una acción. Básicamente a medida que agregamos más código, simplemente estamos actualizando el proceso que va a ejecutar.

In [0]:
import numpy as np

random_numbers = np.random.normal(loc=1000, scale=150, size=10000).tolist()

random_numbers_rdd = sc.parallelize(random_numbers)
random_numbers_rdd

Out[4]: ParallelCollectionRDD[3] at readRDDFromInputStream at PythonRDD.scala:413

In [0]:
random_numbers_rdd.collect()[:50]

Out[5]: [784.9407352833703,
 707.7496474858351,
 1195.2154700158694,
 1131.0838232790516,
 843.8160530493222,
 1036.6981907145569,
 956.4029041353562,
 1004.8710665618433,
 1158.1107622292639,
 1108.5410748919955,
 1021.6217941591485,
 842.347993215826,
 995.9406128501907,
 1021.6740234471407,
 1066.9865240780323,
 1072.8878285262283,
 1020.032123566937,
 1040.978108148427,
 771.4699436752312,
 1026.176426808464,
 902.783537383967,
 895.8443848020192,
 1093.543823260567,
 1203.08699609016,
 856.4279002509276,
 983.5120212383168,
 1032.840558328135,
 1173.5237929789912,
 719.5137505905798,
 1044.4374547805862,
 1166.8134429526394,
 954.3474998295187,
 1088.4126481604771,
 1279.805806534179,
 1023.6889568024322,
 949.7611007516751,
 960.2491111161706,
 727.1922125700114,
 1347.88148389944,
 1188.6733514186008,
 881.9924116046368,
 938.261025120594,
 932.9845215016783,
 919.9831558770751,
 1031.3437186218978,
 718.2971208733121,
 989.4138559008102,
 741.4312901891844,
 831.7258668297848,
 1132.71130284961]

In [0]:
flights

Out[6]: DataFrame[mon: int, dom: int, dow: int, carrier: string, flight: int, org: string, mile: int, depart: double, duration: int, delay: int]

### Crear columnas

Actualizar un Spark DataFrame es algo diferente a trabajar `Pandas` porque el Spark DataFrame es inmutable . Esto significa que no se puede cambiar y, por lo tanto, las columnas no se pueden actualizar en su lugar.

Por lo tanto, todos estos métodos devuelven un nuevo DataFrame.

En Spark puedes hacer esto usando el método `.withColumn()`, que toma dos parámetros. Primero, una cadena con el nombre de su nueva columna y, en segundo lugar, la nueva columna en sí.

La nueva columna debe ser un objeto de clase `Column`. Crear uno de estos es tan fácil como extraer una columna de su DataFrame usando `df.colName`.

In [0]:
flights = flights.withColumn('km', flights.mile * 1.60934)
flights = flights.withColumn('retraso', (flights.delay >= 15).cast("integer"))

flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+----------+-------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km|retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+-------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| null|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30| 508.55144| 1|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8| 542.34758| 0|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5|1989.14424| 0|
 4| 2| 5| AA| 325|ORD| 258| 8.92| 65| null| 415.20972| null|
+---+---+---+-------+------+---+----+------+--------+-----+----------+-------+
only showing top 5 rows

En algunas ocasiones querrás cambiar algún valor o una variable basado en un condicional. Spark, proporciona algunas sentencias condicionales que actúan de forma similar a **_if_**, **_then_**, y **_else_**. Es posible realizar una declaración tradicional del estilo mencionado anteriormente, sin embargo, puede degradar el rendimiento.
Podemos usar entonces condicionales incorporados optimizados, tales como:
* `when(<if condition>, <then x>)`
* `otherwise()`

In [0]:
flights = flights.withColumn("retraso", when(flights.retraso == 1, "demorado")
                             .when(flights.retraso == 0, "a tiempo")
                             .otherwise("N/A"))
flights.show(4)

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30| 508.55144|demorado|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8| 542.34758|a tiempo|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5|1989.14424|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
only showing top 4 rows

In [0]:
flights.withColumnRenamed("retraso", "descripcion").show(4)

+---+---+---+-------+------+---+----+------+--------+-----+----------+-----------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km|descripcion|
+---+---+---+-------+------+---+----+------+--------+-----+----------+-----------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30| 508.55144| demorado|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8| 542.34758| a tiempo|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5|1989.14424| a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+-----------+
only showing top 4 rows

### Filtrar datos

Podemos usar el método `.filter()` para filtrar datos. Este método toma una expresión que seguirá la `WHERE` sentencia de una expresión SQL como un string, o una columna Spark con valores booleanos (`True`/`False`).

**Ejemplo**:

```

df.filter("column > 150").show()

df.filter(df.Column > 150).show()

```

In [0]:
flights.filter("carrier = 'US'").show()

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 5| 30| 1| US| 58|SMF| 397| 20.67| 83| -7| 638.90798|a tiempo|
 1| 13| 3| US| 2189|LGA| 214| 20.0| 82| null| 344.39876| N/A|
 7| 23| 6| US| 49|SFO| 414| 9.38| 92| -9| 666.26676|a tiempo|
 9| 8| 3| US| 2187|LGA| 214| 19.0| 77| null| 344.39876| N/A|
 1| 7| 4| US| 10|ORD|1440| 18.83| 233| 39| 2317.4496|demorado|
 11| 17| 3| US| 870|ORD|1515| 18.83| 237| null| 2438.1501| N/A|
 8| 17| 3| US| 2163|LGA| 214| 7.0| 69| 0| 344.39876|a tiempo|
 3| 21| 1| US| 514|SJC| 386| 13.83| 85| -12| 621.20524|a tiempo|
 6| 2| 3| US| 2144|LGA| 185| 21.0| 72| 1| 297.7279|a tiempo|
 2| 25| 2| US| 2140|LGA| 185| 19.0| 76| -6| 297.7279|a tiempo|
 2| 14| 5| US| 2138|LGA| 185| 18.0| 74| 16| 297.7279|demorado|
 6| 2| 3| US| 2130|LGA| 185| 14.0| 71| 147| 297.7279|demorado|
 0| 5| 6| US| 607|ORD|1440| 5.0| 220| 9| 2317.4496|a tiempo|
 10| 2| 0| US| 384|SMF| 397| 10.13| 81| 47| 638.90798|demorado|
 6| 9| 3| US| 2191|LGA| 214| 21.0| 78| 66| 344.39876|demorado|
 3| 14| 1| US| 31|OGG|2845| 22.67| 349| -19| 4578.5723|a tiempo|
 1| 7| 4| US| 2114|LGA| 185| 6.0| 60| -15| 297.7279|a tiempo|
 4| 3| 6| US| 90|TUS| 110| 11.75| 49| -14| 177.0274|a tiempo|
 10| 3| 1| US| 909|ORD| 599| 7.75| 114| -10| 963.99466|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
only showing top 20 rows

In [0]:
flights.filter(flights.carrier == "US").show(4)

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 5| 30| 1| US| 58|SMF| 397| 20.67| 83| -7| 638.90798|a tiempo|
 1| 13| 3| US| 2189|LGA| 214| 20.0| 82| null| 344.39876| N/A|
 7| 23| 6| US| 49|SFO| 414| 9.38| 92| -9| 666.26676|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
only showing top 4 rows

In [0]:
flights.createOrReplaceTempView("flights")

In [0]:
spark.catalog.listTables()

Out[14]: [Table(name='ciudades', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='diamonds', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='ejemplo_json', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='iris', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='stations', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
query = """
        SELECT *
        FROM flights
        WHERE carrier = 'US'
        """

spark.sql(query).show()

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 5| 30| 1| US| 58|SMF| 397| 20.67| 83| -7| 638.90798|a tiempo|
 1| 13| 3| US| 2189|LGA| 214| 20.0| 82| null| 344.39876| N/A|
 7| 23| 6| US| 49|SFO| 414| 9.38| 92| -9| 666.26676|a tiempo|
 9| 8| 3| US| 2187|LGA| 214| 19.0| 77| null| 344.39876| N/A|
 1| 7| 4| US| 10|ORD|1440| 18.83| 233| 39| 2317.4496|demorado|
 11| 17| 3| US| 870|ORD|1515| 18.83| 237| null| 2438.1501| N/A|
 8| 17| 3| US| 2163|LGA| 214| 7.0| 69| 0| 344.39876|a tiempo|
 3| 21| 1| US| 514|SJC| 386| 13.83| 85| -12| 621.20524|a tiempo|
 6| 2| 3| US| 2144|LGA| 185| 21.0| 72| 1| 297.7279|a tiempo|
 2| 25| 2| US| 2140|LGA| 185| 19.0| 76| -6| 297.7279|a tiempo|
 2| 14| 5| US| 2138|LGA| 185| 18.0| 74| 16| 297.7279|demorado|
 6| 2| 3| US| 2130|LGA| 185| 14.0| 71| 147| 297.7279|demorado|
 0| 5| 6| US| 607|ORD|1440| 5.0| 220| 9| 2317.4496|a tiempo|
 10| 2| 0| US| 384|SMF| 397| 10.13| 81| 47| 638.90798|demorado|
 6| 9| 3| US| 2191|LGA| 214| 21.0| 78| 66| 344.39876|demorado|
 3| 14| 1| US| 31|OGG|2845| 22.67| 349| -19| 4578.5723|a tiempo|
 1| 7| 4| US| 2114|LGA| 185| 6.0| 60| -15| 297.7279|a tiempo|
 4| 3| 6| US| 90|TUS| 110| 11.75| 49| -14| 177.0274|a tiempo|
 10| 3| 1| US| 909|ORD| 599| 7.75| 114| -10| 963.99466|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
only showing top 20 rows

### Seleccionar

La variante Spark para seleccionar es el método `.select()`. Este método toma varios argumentos, uno para cada columna que desee seleccionar. Estos argumentos pueden ser el nombre de la columna como una cadena(uno para cada columna) o un objeto de columna(usando `df.colName`). Cuando pasa un objeto de columna, puede realizar operaciones como sumar o restar en la columna para cambiar los datos que contiene, al igual que en el interior `.withColumn()`.

La diferencia entre los métodos `.select()` y `.withColumn()` es que `.select()` devuelve solo las columnas que especifique, mientras que `.withColumn()` devuelve todas las columnas del DataFrame además de la que definió. A menudo es una buena idea dejar columnas que no necesita al comienzo de una operación para no arrastrar datos adicionales mientras hace cálculos. En este caso, usaría `.select()` y no `.withColumn()`.

**Nota**:

* Cuando hacemos un cálculo o una operación dentro de un `.select()`, para nombrar el resultado podemos usar el método `.alias()`.
* Podemos usar un método `.selectExpr()` que tome expresiones SQL como un string.

In [0]:
flights.select("mile", "km", (flights.mile * 1.60934).alias("km_2")).show(6)

+----+----------+----------+
mile| km| km_2|
+----+----------+----------+
2153|3464.90902|3464.90902|
 316| 508.55144| 508.55144|
 337| 542.34758| 542.34758|
1236|1989.14424|1989.14424|
 258| 415.20972| 415.20972|
 550| 885.137| 885.137|
+----+----------+----------+
only showing top 6 rows

In [0]:
flights.select(flights.mile, flights.km).show(4)

+----+----------+
mile| km|
+----+----------+
2153|3464.90902|
 316| 508.55144|
 337| 542.34758|
1236|1989.14424|
+----+----------+
only showing top 4 rows

### Funciones definidas por el usuario (UDF)

Es un método de Python que el usuario escribe para realizar un bit específico de lógica. Una vez escrito, el método se llama a través de `pyspark.sql.functions.udf`. El resultado se almacena como una variable y se puede llamar como una función de Spark normal.

**Ejemplo**:

```

def reverseString(mystr):
    return mystr[::-1]

udfReverseString = udf(reverseString, StringType())

```

La función `udf(func, dataType)` toma dos argumentos, la función que acaba de definir y el tipo de dato que Spark devolverá.

In [0]:
def lowercase(string):
  return string.lower()

udf_lowercase = udf(lowercase, StringType())

In [0]:
flights.withColumn("carrier", udf_lowercase(flights.carrier)).show(4)

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| us| 19|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 0| 22| 2| ua| 1107|ORD| 316| 16.33| 82| 30| 508.55144|demorado|
 2| 20| 4| ua| 226|SFO| 337| 6.17| 82| -8| 542.34758|a tiempo|
 9| 13| 1| aa| 419|ORD|1236| 10.33| 195| -5|1989.14424|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+
only showing top 4 rows

### Agrupación

Métodos de agrupación como `.min()`, `.max()`, `.count()`, entre otros, se conocen como `GroupedData`. Estos se pueden usar después de usar el método `.groupBy()`.

**Ejemplo**

```

df.groupBy("column").min("column").show()

```

Además de los métodos `GroupedData` también podemos usar un método `.agg()`. Este método permite pasar una expresión de columna agregada que usa cualquier de las funciones agregadas del submódulo `pyspark.sql.functions`. Este submódulo contiene muchas funciones, y toma el nombre de una columna.

In [0]:
display(flights.groupBy("carrier").count())

carrier,count
UA,13170
AA,11316
B6,5267
OO,8148
US,2740
AQ,90
OH,3229
HA,707
WN,5333


In [0]:
flights.groupby("mon", "org").agg(F.min("duration").alias("min_duration"), F.mean("km").alias("avg_km")).show()

+---+---+------------+------------------+
mon|org|min_duration| avg_km|
+---+---+------------+------------------+
 4|ORD| 45|1355.8203923275844|
 6|OGG| 31|1680.3521274999994|
 5|LGA| 64|1245.3944645833328|
 10|TUS| 60| 1186.485915|
 8|TUS| 60|1070.9934180555554|
 10|ORD| 43|1348.7049744091692|
 3|OGG| 31|1577.5134999999984|
 4|SFO| 38|1486.6610453211008|
 1|ORD| 48| 1347.829461064841|
 4|TUS| 49| 989.7772822680422|
 7|SFO| 39|1567.5849768920561|
 9|ORD| 44|1317.9168649434564|
 8|JFK| 66|1775.4374166024459|
 6|SJC| 55|1060.6627481784408|
 2|SMF| 45| 940.7663153612167|
 0|SJC| 55| 993.0697128903661|
 1|JFK| 59|1911.1725665761674|
 11|LGA| 57|1204.2391995979913|
 8|OGG| 31|1460.1082008571423|
 8|SJC| 55| 1051.486556825398|
+---+---+------------+------------------+
only showing top 20 rows

### Join

Para unir DataFrames podemos usar el método `.join()`. Este método recibe tres argumentos, el primero es el DataFrame con el cual voy a unir, el segundo, `on` es el nombre de la(s) columna(s) por las cual voy a unir; y el tercero, el argumento `how` especifica el tipo de join a realizar.

Por otro lado, también puedo usar Spark SQL para unir dos DataFrames. Para eso registramos los DataFrames como tablas en Spark.

**Ejemplo**:

```
df1.createOrRemplaceTempView("df1")
df2.createOrRemplaceTempView("df2")

sql = """
      SELECT *
      FROM df1
      LEFT JOIN df2
      ON df1.column = df2.column
      AND df1.column1 = df2.column1
      """

df = spark.sql(sql)
```

### Eliminar columna(s)

Para eliminar columna(s) en Spark podemos usar el método `.drop()`. Depende de si es una columna o varias, podemos hacerlo de la siguiente manera:

```
# Una sola columna

df = df.drop("Column")

# Varias columnas

cols_to_drop = ["col1", "col2"]

df = df.drop(*cols_to_drop)

```

In [0]:
flights.drop("flight").show(5)

+---+---+---+-------+---+----+------+--------+-----+----------+--------+
mon|dom|dow|carrier|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+---+----+------+--------+-----+----------+--------+
 11| 20| 6| US|JFK|2153| 9.48| 351| null|3464.90902| N/A|
 0| 22| 2| UA|ORD| 316| 16.33| 82| 30| 508.55144|demorado|
 2| 20| 4| UA|SFO| 337| 6.17| 82| -8| 542.34758|a tiempo|
 9| 13| 1| AA|ORD|1236| 10.33| 195| -5|1989.14424|a tiempo|
 4| 2| 5| AA|ORD| 258| 8.92| 65| null| 415.20972| N/A|
+---+---+---+-------+---+----+------+--------+-----+----------+--------+
only showing top 5 rows

In [0]:
flights.drop(*["mile", "km"]).show(5)

+---+---+---+-------+------+---+------+--------+-----+--------+
mon|dom|dow|carrier|flight|org|depart|duration|delay| retraso|
+---+---+---+-------+------+---+------+--------+-----+--------+
 11| 20| 6| US| 19|JFK| 9.48| 351| null| N/A|
 0| 22| 2| UA| 1107|ORD| 16.33| 82| 30|demorado|
 2| 20| 4| UA| 226|SFO| 6.17| 82| -8|a tiempo|
 9| 13| 1| AA| 419|ORD| 10.33| 195| -5|a tiempo|
 4| 2| 5| AA| 325|ORD| 8.92| 65| null| N/A|
+---+---+---+-------+------+---+------+--------+-----+--------+
only showing top 5 rows

## Preprocesamiento

### Valores missing

En muchas ocasiones el set de datos tiene valores missing que tienen algún mecanismo en particular. Podemos usar varios métodos para detectar y tratar los valores missing:

* **.isNull()**: Devuelve `True` cuando la observación es un missing.
* **.fillna(value, subset=None)**: Para reemplazar datos missing por un valor definido. El primer parámetro es el valor a reemplazar, y el segundo, la lista de las columnas en donde se va a imputar el dato missing.
* **.dropna(how="any", thresh=None, subset=None)**: Para eliminar datos missing

In [0]:
for column in flights.columns:
  print(f"La variable {column} tiene {flights.where(flights[column].isNull()).count()} datos missing")

La variable mon tiene 0 datos missing
La variable dom tiene 0 datos missing
La variable dow tiene 0 datos missing
La variable carrier tiene 0 datos missing
La variable flight tiene 0 datos missing
La variable org tiene 0 datos missing
La variable mile tiene 0 datos missing
La variable depart tiene 0 datos missing
La variable duration tiene 0 datos missing
La variable delay tiene 2978 datos missing
La variable km tiene 0 datos missing
La variable retraso tiene 0 datos missing

In [0]:
flights.fillna(0).show()

+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| 0| 3464.90902| N/A|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30| 508.55144|demorado|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8| 542.34758|a tiempo|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5| 1989.14424|a tiempo|
 4| 2| 5| AA| 325|ORD| 258| 8.92| 65| 0| 415.20972| N/A|
 5| 2| 1| UA| 704|SFO| 550| 7.98| 102| 2| 885.137|a tiempo|
 7| 2| 6| AA| 380|ORD| 733| 10.83| 135| 54| 1179.64622|demorado|
 1| 16| 6| UA| 1477|ORD|1440| 8.0| 232| -7| 2317.4496|a tiempo|
 1| 22| 5| UA| 620|SJC|1829| 7.98| 250| -13| 2943.48286|a tiempo|
 11| 8| 1| OO| 5590|SFO| 158| 7.77| 60| 88| 254.27572|demorado|
 4| 26| 1| AA| 1144|SFO|1464| 13.25| 210| -10|2356.0737599999998|a tiempo|
 4| 25| 0| AA| 321|ORD| 978| 13.75| 160| 31| 1573.93452|demorado|
 8| 30| 2| UA| 646|ORD| 719| 13.28| 151| 16| 1157.11546|demorado|
 3| 16| 3| UA| 107|ORD|1745| 9.0| 264| 3| 2808.2983|a tiempo|
 0| 3| 4| AA| 1559|LGA|1097| 17.08| 190| 32| 1765.44598|demorado|
 5| 9| 1| UA| 770|SFO| 967| 12.7| 158| 20| 1556.23178|demorado|
 3| 10| 4| B6| 937|ORD|1735| 17.58| 265| 155| 2792.2049|demorado|
 11| 15| 1| AA| 2303|ORD| 802| 6.75| 160| 23| 1290.69068|demorado|
 8| 18| 4| UA| 802|SJC| 948| 6.33| 160| 17| 1525.65432|demorado|
 2| 14| 5| B6| 71|JFK| 944| 6.17| 166| 0| 1519.21696|a tiempo|
+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
only showing top 20 rows

In [0]:
flights.dropna().show()

+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|
+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30| 508.55144|demorado|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8| 542.34758|a tiempo|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5| 1989.14424|a tiempo|
 5| 2| 1| UA| 704|SFO| 550| 7.98| 102| 2| 885.137|a tiempo|
 7| 2| 6| AA| 380|ORD| 733| 10.83| 135| 54| 1179.64622|demorado|
 1| 16| 6| UA| 1477|ORD|1440| 8.0| 232| -7| 2317.4496|a tiempo|
 1| 22| 5| UA| 620|SJC|1829| 7.98| 250| -13| 2943.48286|a tiempo|
 11| 8| 1| OO| 5590|SFO| 158| 7.77| 60| 88| 254.27572|demorado|
 4| 26| 1| AA| 1144|SFO|1464| 13.25| 210| -10|2356.0737599999998|a tiempo|
 4| 25| 0| AA| 321|ORD| 978| 13.75| 160| 31| 1573.93452|demorado|
 8| 30| 2| UA| 646|ORD| 719| 13.28| 151| 16| 1157.11546|demorado|
 3| 16| 3| UA| 107|ORD|1745| 9.0| 264| 3| 2808.2983|a tiempo|
 0| 3| 4| AA| 1559|LGA|1097| 17.08| 190| 32| 1765.44598|demorado|
 5| 9| 1| UA| 770|SFO| 967| 12.7| 158| 20| 1556.23178|demorado|
 3| 10| 4| B6| 937|ORD|1735| 17.58| 265| 155| 2792.2049|demorado|
 11| 15| 1| AA| 2303|ORD| 802| 6.75| 160| 23| 1290.69068|demorado|
 8| 18| 4| UA| 802|SJC| 948| 6.33| 160| 17| 1525.65432|demorado|
 2| 14| 5| B6| 71|JFK| 944| 6.17| 166| 0| 1519.21696|a tiempo|
 7| 21| 4| OO| 5806|ORD| 607| 19.0| 110| 21| 976.86938|demorado|
 11| 6| 6| OO| 5524|SFO| 316| 8.75| 82| 40| 508.55144|demorado|
+---+---+---+-------+------+---+----+------+--------+-----+------------------+--------+
only showing top 20 rows

In [0]:
train_data, test_data = flights.randomSplit([0.7, 0.3], seed=42)

Recordemos que en sklearn, tenemos:

* Estimadores .fit()
* Transformadores .transform()
* Predictores .predict()

En PySpark.ml, tenemos:

* Estimadores .fit()
* Transformadores .transform()

In [0]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy="mean", inputCol="delay", outputCol="delay_imp")

imputer = imputer.fit(train_data)

train_data = imputer.transform(train_data)

test_data = imputer.transform(test_data)


+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|delay_imp|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+
 0| 1| 2| AA| 3|JFK|2475| 12.0| 370| 11| 3983.1165|a tiempo| 11|
 0| 1| 2| AA| 59|JFK|2586| 7.0| 385| -16|4161.75324|a tiempo| -16|
 0| 1| 2| AA| 150|SFO|2704| 23.42| 325| 22|4351.65536|demorado| 22|
 0| 1| 2| AA| 154|ORD| 867| 17.25| 135| 49|1395.29778|demorado| 49|
 0| 1| 2| AA| 181|JFK|2475| 17.0| 379| -10| 3983.1165|a tiempo| -10|
 0| 1| 2| AA| 317|LGA| 733| 9.92| 170| -9|1179.64622|a tiempo| -9|
 0| 1| 2| AA| 335|LGA| 733| 14.58| 165| -4|1179.64622|a tiempo| -4|
 0| 1| 2| AA| 336|ORD| 733| 21.58| 115| 55|1179.64622|demorado| 55|
 0| 1| 2| AA| 346|ORD| 733| 19.5| 130| 63|1179.64622|demorado| 63|
 0| 1| 2| AA| 392|ORD| 733| 8.08| 120| 4|1179.64622|a tiempo| 4|
 0| 1| 2| AA| 502|SJC|1476| 17.58| 200| 66|2375.38584|demorado| 66|
 0| 1| 2| AA| 510|ORD| 235| 19.0| 80| 61| 378.1949|demorado| 61|
 0| 1| 2| AA| 678|SFO| 337| 16.25| 80| 139| 542.34758|demorado| 139|
 0| 1| 2| AA| 709|LGA|1389| 8.25| 250| 27|2235.37326|demorado| 27|
 0| 1| 2| AA| 895|ORD| 888| 12.67| 160| 68|1429.09392|demorado| 68|
 0| 1| 2| AA| 1184|ORD| 719| 12.0| 125| null|1157.11546| N/A| 28|
 0| 1| 2| AA| 1201|ORD| 258| 15.0| 70| 62| 415.20972|demorado| 62|
 0| 1| 2| AA| 1561|ORD|1846| 18.58| 275| 65|2970.84164|demorado| 65|
 0| 1| 2| AA| 1709|LGA|1097| 20.42| 185| 31|1765.44598|demorado| 31|
 0| 1| 2| AA| 1904|ORD|1197| 17.75| 180| 237|1926.37998|demorado| 237|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+
only showing top 20 rows

### Variables categóricas

Algunas clases que nos ayudan a codificar datos categóricos en la etapa de preprocesamiento son:

* `StringIndexer(inputCol, outputCol)` Los valores de índice que se asignan es basado en la frecuencia relativa de forma descendente.
* `OneHotEncoder()`

In [0]:
indexer = StringIndexer(inputCol='org', outputCol="org_idx")

train_data = train_data.drop("org_idx")

model_indexer = indexer.fit(train_data)

In [0]:
train_data = model_indexer.transform(train_data)

test_data = model_indexer.transform(test_data)

In [0]:
ohe = OneHotEncoder(inputCol='org_idx', outputCol='org_dummy').fit(train_data)

train_data = ohe.transform(train_data)

test_data = ohe.transform(test_data)

### Creación de vector predictor (Assembling)

En Spark se debe consolidar las distantes variables independientes en una sola columna

* `VectorAssembler(inputCols, outputCol)`

In [0]:
train_data.show(4)

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+-------+-------------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|delay_imp|org_idx| org_dummy|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+-------+-------------+
 0| 1| 2| AA| 3|JFK|2475| 12.0| 370| 11| 3983.1165|a tiempo| 11| 2.0|(7,[2],[1.0])|
 0| 1| 2| AA| 59|JFK|2586| 7.0| 385| -16|4161.75324|a tiempo| -16| 2.0|(7,[2],[1.0])|
 0| 1| 2| AA| 150|SFO|2704| 23.42| 325| 22|4351.65536|demorado| 22| 1.0|(7,[1],[1.0])|
 0| 1| 2| AA| 154|ORD| 867| 17.25| 135| 49|1395.29778|demorado| 49| 0.0|(7,[0],[1.0])|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+---------+-------+-------------+
only showing top 4 rows

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['org_dummy', 'km'], outputCol='features')

In [0]:
train_data = assembler.transform(train_data)

test_data = assembler.transform(test_data)

In [0]:
test_data.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|org_idx| org_dummy| features|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+
 0| 1| 2| AA| 73|ORD|4243| 9.08| 560| 39|6828.42962|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,682...|
 0| 1| 2| AA| 254|OGG|2486| 15.33| 310| 173|4000.81924|demorado| 7.0| (7,[],[])|(8,[7],[4000.81924])|
 0| 1| 2| AA| 321|ORD| 334| 14.17| 90| null| 537.51956| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,537...|
 0| 1| 2| AA| 325|ORD| 258| 10.0| 70| null| 415.20972| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,415...|
 0| 1| 2| AA| 354|ORD| 733| 17.58| 130| 28|1179.64622|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,117...|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+
only showing top 5 rows

## Entrenamiento

In [0]:
linear_reg = LinearRegression(labelCol='duration', featuresCol='features').fit(train_data)

In [0]:
predictions = linear_reg.transform(test_data)
predictions.show()

+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+------------------+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay| km| retraso|org_idx| org_dummy| features| prediction|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+------------------+
 0| 1| 2| AA| 73|ORD|4243| 9.08| 560| 39|6828.42962|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,682...| 551.279921972351|
 0| 1| 2| AA| 254|OGG|2486| 15.33| 310| 173|4000.81924|demorado| 7.0| (7,[],[])|(8,[7],[4000.81924])|312.86636673360283|
 0| 1| 2| AA| 321|ORD| 334| 14.17| 90| null| 537.51956| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,537...| 84.2763878112149|
 0| 1| 2| AA| 325|ORD| 258| 10.0| 70| null| 415.20972| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,415...| 75.19675910918207|
 0| 1| 2| AA| 354|ORD| 733| 17.58| 130| 28|1179.64622|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,117...|131.94443849688727|
 0| 1| 2| AA| 366|ORD| 733| 14.5| 135| 52|1179.64622|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,117...|131.94443849688727|
 0| 1| 2| AA| 386|ORD| 733| 8.75| 120| 8|1179.64622|a tiempo| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,117...|131.94443849688727|
 0| 1| 2| AA| 547|ORD|1652| 10.17| 251| 67|2658.62968|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,265...|241.73626451225797|
 0| 1| 2| AA| 705|LGA|1389| 6.5| 240| 40|2235.37326|demorado| 3.0|(7,[3],[1.0])|(8,[3,7],[1.0,223...|228.59044150157015|
 0| 1| 2| AA| 769|JFK|1597| 6.58| 230| 50|2570.11598|demorado| 2.0|(7,[2],[1.0])|(8,[2,7],[1.0,257...| 259.1975443487884|
 0| 1| 2| AA| 860|ORD| 235| 21.92| 64| 62| 378.1949|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,378...| 72.44897673883003|
 0| 1| 2| AA| 1314|TUS| 813| 10.58| 130| 75|1308.39342|demorado| 6.0|(7,[6],[1.0])|(8,[6,7],[1.0,130...|130.87195231228995|
 0| 1| 2| AA| 1345|ORD|1745| 14.08| 270| null| 2808.2983| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,280...| 252.8468627923771|
 0| 1| 2| AA| 1472|SFO|1846| 13.67| 259| 7|2970.84164|a tiempo| 1.0|(7,[1],[1.0])|(8,[1,7],[1.0,297...| 256.9836576008864|
 0| 1| 2| AA| 1659|ORD| 403| 21.08| 85| 29| 648.56402|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,648...| 92.51973492227103|
 0| 1| 2| AA| 1763|ORD|1726| 16.75| 260| 137|2777.72084|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,277...|250.57695561686887|
 0| 1| 2| AA| 1797|ORD| 258| 11.67| 65| null| 415.20972| N/A| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,415...| 75.19675910918207|
 0| 1| 2| AA| 2060|SMF|1431| 14.0| 200| 3|2302.96554|a tiempo| 5.0|(7,[5],[1.0])|(8,[5,7],[1.0,230...|202.41539181807423|
 0| 1| 2| AA| 2085|ORD| 403| 16.92| 90| 101| 648.56402|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,648...| 92.51973492227103|
 0| 1| 2| AA| 2303|ORD| 802| 6.75| 150| 102|1290.69068|demorado| 0.0|(7,[0],[1.0])|(8,[0,7],[1.0,129...| 140.1877856079434|
+---+---+---+-------+------+---+----+------+--------+-----+----------+--------+-------+-------------+--------------------+------------------+
only showing top 20 rows

In [0]:
RegressionEvaluator(labelCol='duration').evaluate(predictions)

Out[54]: 11.490330518273224

In [0]:
linear_reg.coefficients

Out[55]: DenseVector([28.5069, 20.5773, 52.5389, 46.7813, 18.3355, 15.5886, 17.8769, 0.0742])

In [0]:
linear_reg.intercept

Out[56]: 15.866933138160427